In [1]:
import json
import requests
from pprint import pprint
from config import api_key
import pandas as pd

In [2]:
#Setting up the URL to request the list of sessions from
url = "	https://api.legiscan.com/"
api_key = "?key="+api_key

In [3]:
# Getting the list of available datasets
list_json = requests.get(url + api_key + "&op=getDatasetList&state=VA").json()
pprint(json.dumps(list_json, indent=4, sort_keys=False))

('{\n'
 '    "status": "OK",\n'
 '    "datasetlist": [\n'
 '        {\n'
 '            "state_id": 46,\n'
 '            "session_id": 1988,\n'
 '            "year_start": 2023,\n'
 '            "year_end": 2023,\n'
 '            "prefile": 0,\n'
 '            "sine_die": 0,\n'
 '            "prior": 0,\n'
 '            "special": 0,\n'
 '            "session_tag": "Regular Session",\n'
 '            "session_title": "2023 Regular Session",\n'
 '            "session_name": "2023 Regular Session",\n'
 '            "dataset_date": "2023-03-26",\n'
 '            "dataset_hash": "914eddc724c0f871c1fd295530814d6b",\n'
 '            "dataset_size": 11153065,\n'
 '            "access_key": "3AyhWDxVPWjbjiy7yofuWZ"\n'
 '        },\n'
 '        {\n'
 '            "state_id": 46,\n'
 '            "session_id": 1975,\n'
 '            "year_start": 2022,\n'
 '            "year_end": 2022,\n'
 '            "prefile": 0,\n'
 '            "sine_die": 1,\n'
 '            "prior": 1,\n'
 '            "s

In [ ]:
#Collecting the session info into lists that I will use to create a dataframe
session_id_list=[]
session_title_list=[]
access_key_list=[]
year_list=[]
#No session goes between years so I am just going to use the year end start date as a way of grouping years for later use
for session in list_json['datasetlist']:
    session_id_list.append(session['session_id'])
    session_title_list.append(session['session_title'])
    access_key_list.append(session['access_key'])
    year_list.append(session['year_start'])
    
    

In [ ]:
# Creating a dataframe that contains the session info to loop through
sessions_df=pd.DataFrame({"Session_ID": session_id_list,
                          "Session Year": year_list,
                          "Session Title": session_title_list,
                          "Session Access Key": access_key_list
                         })
sessions_df

In [ ]:
# Creating the url format to make my requests
master_list_url = url + api_key + "&op=getMasterList&id="

In [ ]:
# Creating lists to fill with my desired info on the bills from each session
bills=[]
bill_number=[]
bill_id=[]
bill_status=[]
description=[]

In [ ]:
# Creating a loop that collects all the bills 
for index, row in sessions_df.iterrows():
    session_json = requests.get(master_list_url + str(row["Session_ID"])).json()
    bill_list=session_json["masterlist"]
    for key in bill_list:
        if key == "session":
            continue
        bill = bill_list[key]
        bills.append(bill['title'])
        bill_number.append(bill['number'])
        bill_id.append(bill['bill_id'])
        bill_status.append(bill['status'])
        description.append(bill['description'])

In [ ]:
# Generating a dataframe from the collected data
all_bills=pd.DataFrame({
                        "Bill ID": bill_id,
                        "Bill Number": bill_number,
                        "Bill Title": bills,
                        "Bill Status": bill_status,
                        "Bill Description": description
                        
    
})    

all_bills

In [ ]:
all_bills.to_csv('virginia_bills_df.csv')

In [ ]:
# Creating a search function to search for bills with certain keywords in their descriptions
all_bills.loc[all_bills['Bill Description'].str.contains("involuntary commitment", case=False)]